In [1]:
# from qiskit.opflow import I, Z, X, Y, SparsePauliOp
import numpy as np
# from juices import Juices
# 假设有 3 台机器和 3 个操作
m = 3 # 机器数量
N = 2 # 表示机器的二进制 (0, 1)
n = 2 # 操作数量
l = 1  # 用于表示 C_max 的二进制位数

# 处理时间矩阵 (例如)
p = np.array([[1, 2, 3],[3, 1, 2]])

# 非法数组(用于表示机器数)(1, 1)
b = np.array([1, 1])

# 常数
K1 = 10
P = 5

# 量子比特数量
num_qubits = n * N + l